# Análise de Dados - Olist

Este jupyter notebook apresenta uma análise geral dos dados da plataforma de vendas Olist, organizando e visualizando informações sobre clientes, produtos e vendas.

### 1. Importação das Bibliotecas e Leitura dos Dados

Carregamos os arquivos CSV para os respectivos DataFrames.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)

caminho_arquivos = {
    "clientes": "dados/olist_customers_dataset.csv",
    "geolocalizacao": "dados/olist_geolocation_dataset.csv",
    "itens_pedidos": "dados/olist_order_items_dataset.csv",
    "pagamentos_pedidos": "dados/olist_order_payments_dataset.csv",
    "avaliacoes_pedidos": "dados/olist_order_reviews_dataset.csv",
    "pedidos": "dados/olist_orders_dataset.csv",
    "produtos": "dados/olist_products_dataset.csv",
    "vendedores": "dados/olist_sellers_dataset.csv",
    "traducao_categorias_produtos": "dados/product_category_name_translation.csv",
}

dataframes = {nome: pd.read_csv(caminho) for nome, caminho in caminho_arquivos.items()}

clientes_df = dataframes["clientes"]
geolocalizacao_df = dataframes["geolocalizacao"]
itens_pedidos_df = dataframes["itens_pedidos"]
pagamentos_pedidos_df = dataframes["pagamentos_pedidos"]
avaliacoes_pedidos_df = dataframes["avaliacoes_pedidos"]
pedidos_df = dataframes["pedidos"]
produtos_df = dataframes["produtos"]
vendedores_df = dataframes["vendedores"]
traducao_categorias_produtos_df = dataframes["traducao_categorias_produtos"]

### 2. Pré-visualização dos Dados

Exibimos uma amostra de cada DataFrame para entender sua estrutura e conteúdo.

In [ ]:
clientes_df.sample(2)

In [ ]:
geolocalizacao_df.sample(2)

In [ ]:
itens_pedidos_df.sample(2)

In [ ]:
pagamentos_pedidos_df.sample(2)

In [ ]:
avaliacoes_pedidos_df.sample(2)

In [ ]:
pedidos_df.sample(2)

In [ ]:
produtos_df.sample(2)

In [ ]:
vendedores_df.sample(5)

In [ ]:
traducao_categorias_produtos_df.sample(2)

In [ ]:
produtos_df.describe()

### 4. Tratamento de Dados

#### 4.1 Identificação de Valores Ausentes

Identificamos valores ausentes em cada coluna dos DataFrames e organizamos os resultados em uma tabela consolidada para melhor visualização.

In [ ]:
resumo = []

for nome, df in dataframes.items():
    valores_ausentes = df.isnull().sum() 
    df_resumo = pd.DataFrame({
        "DataFrame": [nome] * len(valores_ausentes),
        "Coluna": valores_ausentes.index,
        "Valores ausentes": valores_ausentes.values
    })
    resumo.append(df_resumo)
    
df_resumo_final = pd.concat(resumo, ignore_index=True)
df_resumo_final


#### 4.2 Substituição de Valores Ausentes

In [ ]:
avaliacoes_pedidos_df['review_comment_message'] = avaliacoes_pedidos_df['review_comment_message'].fillna('Sem Comentário')
avaliacoes_pedidos_df['review_comment_title'] = avaliacoes_pedidos_df['review_comment_title'].fillna('Sem Título')

In [ ]:
# preenche valores ausentes com a média nas colunas numéricas do DataFrame 'produtos_df'
colunas_numericas = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

produtos_df[colunas_numericas] = produtos_df[colunas_numericas].apply(lambda col: col.fillna(col.mean()))

### 5. Análise Exploratória de Dados (EDA)

Exploramos os dados para obter _insights_ sobre tempos de entrega, vendas por categoria e vendas por localização geográfica.

#### 5.1 Tempo Médio de Entrega

Calculamos o tempo médio de entrega com base nas datas de compra e entrega ao cliente:

In [ ]:
pedidos_df['order_delivered_customer_date'] = pd.to_datetime(pedidos_df['order_delivered_customer_date'])
pedidos_df['order_purchase_timestamp'] = pd.to_datetime(pedidos_df['order_purchase_timestamp'])
pedidos_df['delivery_time'] = (pedidos_df['order_delivered_customer_date'] - pedidos_df['order_purchase_timestamp']).dt.days

tempo_medio_entrega = pedidos_df['delivery_time'].mean()
tempo_medio_entrega = round(tempo_medio_entrega, 2)
print(f"Tempo médio de entrega: {tempo_medio_entrega} dias")

#### 5.2 Vendas por Categoria de Produto

Analisamos a frequência de vendas por categoria de produto, integrando os dados de itens de pedidos e produtos.

In [ ]:
itens_produtos = pd.merge(itens_pedidos_df, produtos_df, on='product_id', how='inner')

vendas_por_categoria = itens_produtos['product_category_name'].value_counts()

vendas_por_categoria_df = pd.DataFrame(vendas_por_categoria).reset_index()
vendas_por_categoria_df.columns = ['Categoria de Produto', 'Frequência de Vendas']

vendas_por_categoria_df


#### 5.3 Vendas por Localização Geográfica

Exploramos a distribuição das vendas com base nas informações de localização de clientes e geolocalização.

##### Por estado:

In [ ]:
vendas_geolocalizacao = pd.merge(clientes_df, geolocalizacao_df, 
                                  left_on='customer_zip_code_prefix', 
                                  right_on='geolocation_zip_code_prefix', 
                                  how='inner')

vendas_por_estado = vendas_geolocalizacao['geolocation_state'].value_counts()

vendas_por_estado_df = pd.DataFrame(vendas_por_estado).reset_index()
vendas_por_estado_df.columns = ['Estado', 'Frequência de Vendas']

vendas_por_estado_df


#### Por cidade:

In [ ]:
vendas_por_cidade = vendas_geolocalizacao['geolocation_city'].value_counts()

vendas_por_cidade_df = pd.DataFrame(vendas_por_cidade).reset_index()
vendas_por_cidade_df.columns = ['Cidade', 'Frequência de Vendas']

vendas_por_cidade_df.head(50)
